In [2]:
import boto3

import pandas as pd
import numpy as np

df = pd.read_csv("/Users/mariolaczajkowska/anaconda_projects/python/project_mcz2/downloads/Airbnb_Open_Data.csv", low_memory=False)

# upper columns headers
df.columns = df.columns.str.upper()

# replaced ' ' for _
df.columns = df.columns.str.replace(' ', '_')

# cleaning data (null and exluded columns)

df_cleaned = df.dropna(subset=['MINIMUM_NIGHTS', 'PRICE', 'NAME','SERVICE_FEE', 'COUNTRY', 'INSTANT_BOOKABLE', 'NEIGHBOURHOOD_GROUP', 'HOST_IDENTITY_VERIFIED', 'LAT', 'LONG', 'REVIEW_RATE_NUMBER']).copy() \
               .drop(columns=['NEIGHBOURHOOD', 'COUNTRY', 'AVAILABILITY_365', 'LICENSE', 'REVIEWS_PER_MONTH', 'ID', 'HOST_ID', 'CONSTRUCTION_YEAR', 'COUNTRY_CODE', 'CALCULATED_HOST_LISTINGS_COUNT', 'LAST_REVIEW'])


negative_values = df_cleaned[df_cleaned['MINIMUM_NIGHTS'] < 0]
df_cleaned = df_cleaned.drop(df_cleaned[df_cleaned['MINIMUM_NIGHTS'] < 0].index)
df_cleaned = df_cleaned[~df_cleaned.isin(['#NAME', '#NAME?']).any(axis=1)]
df_cleaned['HOST_IDENTITY_VERIFIED'] = df_cleaned['HOST_IDENTITY_VERIFIED'].str.capitalize()
df_cleaned['CANCELLATION_POLICY'] = df_cleaned['CANCELLATION_POLICY'].str.capitalize()
df_cleaned['INSTANT_BOOKABLE'] = df_cleaned['INSTANT_BOOKABLE'].astype(bool)
df_cleaned['LAT'] = pd.to_numeric(df_cleaned['LAT'], errors='coerce')
df_cleaned['LONG'] = pd.to_numeric(df_cleaned['LONG'], errors='coerce')
df_cleaned = df_cleaned.dropna(subset=['LAT', 'LONG'])
# fill missing values
df_cleaned.insert(3, 'CITY', 'New York')
df_cleaned['HOUSE_RULES'] = df_cleaned['HOUSE_RULES'].fillna('No information')
df_cleaned['HOST_NAME'] = df_cleaned['HOST_NAME'].fillna('Prefer not to say')
df_cleaned = df_cleaned.drop(df_cleaned[df_cleaned['MINIMUM_NIGHTS'] < 0].index)
df_cleaned['MINIMUM_NIGHTS'] = df_cleaned['MINIMUM_NIGHTS'].replace('No minimum stay required', 0)
df_cleaned['MINIMUM_NIGHTS'] = pd.to_numeric(df_cleaned['MINIMUM_NIGHTS'], errors='coerce')  # Ensure numeric type
df_cleaned['NUMBER_OF_REVIEWS'] = df_cleaned['NUMBER_OF_REVIEWS'].fillna(0).astype(int)

df_cleaned['PRICE'] = df_cleaned['PRICE'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)
df_cleaned['PRICE'] = df_cleaned['PRICE'].astype(int)

df_cleaned['SERVICE_FEE']=df['SERVICE_FEE'].str.replace('$', '')
df_cleaned.insert(12, 'CURRENCY', 'USD')
df_cleaned['NEIGHBOURHOOD_GROUP'] = df_cleaned['NEIGHBOURHOOD_GROUP'].replace('brookln', 'Brooklyn')

df_cleaned.to_csv('downloads/cleaned_airbnb_data.csv', index=False)

print(df_cleaned)


                                                    NAME  \
0                     Clean & quiet apt home by the park   
1                                  Skylit Midtown Castle   
4       Entire Apt: Spacious Studio/Loft by central park   
5              Large Cozy 1 BR Apartment In Midtown East   
7                                        BlissArtsSpace!   
...                                                  ...   
102594                        Spare room in Williamsburg   
102595                     Best Location near Columbia U   
102596                    Comfy, bright room in Brooklyn   
102597                  Big Studio-One Stop from Midtown   
102598                              585 sf Luxury Studio   

       HOST_IDENTITY_VERIFIED    HOST_NAME      CITY NEIGHBOURHOOD_GROUP  \
0                 Unconfirmed     Madaline  New York            Brooklyn   
1                    Verified        Jenna  New York           Manhattan   
4                    Verified       Lyndon  New Yor